### Filter
A filter is a reusable piece of code that can transform the content of HTTP requests, responses, and header information. Filters do not generally create a response or respond to a request as servlets do, rather they modify or adapt the requests/ response.  

Filters are generally used for:
- Logging
- Authentication
- Compression
- Encryption
- Caching

### Lifecycle
A filter's lifecycle starts with the `init` method where we can setup set-up tasks before the filter is called. The `doFilter` method is called every time the Container determines that the filter should be applied to the current request. Finally, when the Container decides to remove a filter instance, it calls the `destroy` method.  

A number of filters can be applied to a path. In that case one filter calls the other filter when `chain.doFilter` line is reached. At one point in chain, the filter calls the servlet. Once the servlet responds, the filters are again called. We can visualize the chain in terms of a stack: 

![Filter Chain](https://i.imgur.com/i4tghgb.png)

### Configuring Filter
We can add a filter in web.xml by:

```xml
<filter>
  <filter-name>XSSFilter</filter-name>
  <filter-class>com.example.XSSFilter
        </filter-class>
  <init-param>
    <param-name>LogFileName</param-name>
    <param-value>UserLog.txt</param-value>
  </init-param>
</filter>
```

We can map it to a URL by:
```xml
<filter-mapping>
  <filter-name>XSSFilter</filter-name>
  <url-pattern>*</url-pattern>
</filter-mapping>
```

Or, we can map it to a servlet
```xml
<filter-mapping>
  <filter-name>XSSFilter</filter-name>
  <servlet-name>LoginServlet</servlet-name>
</filter-mapping>
```

The order of filters is the order their mappings are defined in web.xml. If using annotations (@WebFilter) the order seems to be undefined, so do declare filter mapping in web.xml

### Example
**XSS Filter** strips any XSS Script from request

In [ ]:
public class XSSFilter implements Filter {

    @Override
    public void init(FilterConfig filterConfig) throws ServletException {
    }

    @Override
    public void destroy() {
    }

    @Override
    public void doFilter(ServletRequest request, ServletResponse response, FilterChain chain)
        throws IOException, ServletException {
        chain.doFilter(new XSSRequestWrapper((HttpServletRequest) request), response);
    }

}

Create a custom request object by extending `HttpServletRequestWrapper`

In [ ]:
public class XSSRequestWrapper extends HttpServletRequestWrapper {

    private static Pattern[] patterns = new Pattern[] {
            // Script fragments
            Pattern.compile("<script>(.*?)</script>", Pattern.CASE_INSENSITIVE),
            // src='...'
            Pattern.compile("src[\r\n]*=[\r\n]*\\\'(.*?)\\\'",
                    Pattern.CASE_INSENSITIVE | Pattern.MULTILINE | Pattern.DOTALL),
            Pattern.compile("src[\r\n]*=[\r\n]*\\\"(.*?)\\\"",
                    Pattern.CASE_INSENSITIVE | Pattern.MULTILINE | Pattern.DOTALL),
            // lonely script tags
            Pattern.compile("</script>", Pattern.CASE_INSENSITIVE),
            Pattern.compile("<script(.*?)>", Pattern.CASE_INSENSITIVE | Pattern.MULTILINE | Pattern.DOTALL),
            // eval(...)
            Pattern.compile("eval\\((.*?)\\)", Pattern.CASE_INSENSITIVE | Pattern.MULTILINE | Pattern.DOTALL),
            // expression(...)
            Pattern.compile("expression\\((.*?)\\)", Pattern.CASE_INSENSITIVE | Pattern.MULTILINE | Pattern.DOTALL),
            // javascript:...
            Pattern.compile("javascript:", Pattern.CASE_INSENSITIVE),
            // vbscript:...
            Pattern.compile("vbscript:", Pattern.CASE_INSENSITIVE),
            // onload(...)=...
            Pattern.compile("onload(.*?)=", Pattern.CASE_INSENSITIVE | Pattern.MULTILINE | Pattern.DOTALL) };

    public XSSRequestWrapper(HttpServletRequest servletRequest) {
        super(servletRequest);
    }

    @Override
    public String[] getParameterValues(String parameter) {
        String[] values = super.getParameterValues(parameter);

        if (values == null) {
            return null;
        }

        int count = values.length;
        String[] encodedValues = new String[count];
        for (int i = 0; i < count; i++) {
            encodedValues[i] = stripXSS(values[i]);
        }

        return encodedValues;
    }

    @Override
    public String getParameter(String parameter) {
        String value = super.getParameter(parameter);

        return stripXSS(value);
    }

    @Override
    public String getHeader(String name) {
        String value = super.getHeader(name);
        return stripXSS(value);
    }

    private String stripXSS(String value) {
        if (value != null) {
            // NOTE: It's highly recommended to use the ESAPI library and uncomment the
            // following line to avoid encoded attacks.
            // value = ESAPI.encoder().canonicalize(value);

            // Avoid null characters
            value = value.replaceAll("\0", "");

            // Remove all sections that match a pattern
            for (Pattern scriptPattern : patterns) {
                value = scriptPattern.matcher(value).replaceAll("");
            }
        }
        return value;
    }
}


**Compression filter**:

In [ ]:
public class CompressionFilter implements Filter {

    @Override
    public void init(FilterConfig filterConfig) throws ServletException {
    }

    @Override
    public void destroy() {
    }

    @Override
    public void doFilter(ServletRequest request, ServletResponse response, FilterChain chain)
        throws IOException, ServletException {
        CompressServletResponseWrapper compressedResponse = new CompressServletResponseWrapper(response);
        chain.doFilter(request, response);
    }

}

We need to always use a response wrapper if we want to process things after response is obtained from servlet. This is because servlet may commit the response.

### Filters and the RequestDispatcher
Since version 2.4 of the Java Servlet specification is the ability to configure filters to be invoked under request dispatcher `forward` and `include` calls.

For example,
```xml
<filter-mapping>
    <filter-name>Logging Filter</filter-name>
    <url-pattern>/products/*</url-pattern>
</filter-mapping>
```
would result in the Logging Filter being invoked by client requests starting /products/... but not underneath a request dispatcher call where the request dispatcher has path commencing /products/.... 

Another example,
```xml
<filter-mapping>
    <filter-name>Logging Filter</filter-name>
    <servlet-name>ProductServlet</servlet-name>
    <dispatcher>INCLUDE</dispatcher>
</filter-mapping>
```
would result in the Logging Filter not being invoked by client requests to the ProductServlet, nor underneath a request dispatcher `forward` call to the ProductServlet, but would be invoked underneath a request dispatcher `include` call where the request dispatcher has a name commencing ProductServlet.

Finally
```xml
<filter-mapping>
    <filter-name>Logging Filter</filter-name>
    <url-pattern>/products/*</url-pattern>
    <dispatcher>FORWARD</dispatcher>
    <dispatcher>REQUEST</dispatcher>
</filter-mapping>
```
would result in the Logging Filter being invoked by client requests starting /products/... and underneath a request dispatcher `forward` call where the request dispatcher has path commencing /products/....